# Stable Baselines3 - Mountain Car

In [ ]:
!apt-get update -qq && apt-get install ffmpeg freeglut3-dev xvfb -qq  # For visualization
#!pip install box2d-py -q
!pip install -q "stable-baselines3[extra]>=2.0.0a4"

## Importações

In [ ]:
import pandas as pd
import stable_baselines3
import gymnasium as gym
import numpy as np

### Importando Algoritmo e Política

In [ ]:
from stable_baselines3 import DQN
from stable_baselines3.ppo import MlpPolicy

### Função Auxiliar

In [ ]:
def evaluate(model, num_episodes=10, deterministic=True, qi=0):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_episodes: (int) number of episodes to evaluate it
    :return: (float) Mean reward for the last num_episodes
    """
    # This function will only work for a single Environment
    vec_env = model.get_env()
    all_episode_rewards = []
    for i in range(num_episodes):
        episode_rewards = []
        done = False
        obs = vec_env.reset()
        while not done:
            # _states are only useful when using LSTM policies
            action, _states = model.predict(obs, deterministic=deterministic)
            # here, action, rewards and dones are arrays
            # because we are using vectorized env
            # also note that the step only returns a 4-tuple, as the env that is returned
            # by model.get_env() is an sb3 vecenv that wraps the >v0.26 API

            #obs, reward, done, info = vec_env.step(action)
            obs, reward, done, info = vec_env.step(action)

            episode_rewards.append(reward)

            #done = terminated or truncated


        all_episode_rewards.append(sum(episode_rewards))
        df_treinos.loc[qi][i + 1] = sum(episode_rewards)[0]

    mean_episode_reward = np.mean(all_episode_rewards)
    print("Mean reward:", mean_episode_reward, "Num episodes:", num_episodes)

    return mean_episode_reward

### Método de avaliação do Stable Baselines

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

### Criando Ambiente

In [ ]:
env = gym.make("MountainCar-v0", render_mode="rgb_array" )

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Espaço de observação
env.observation_space

Box([-1.2  -0.07], [0.6  0.07], (2,), float32)

In [ ]:
# Espaço de ações
env.action_space

Discrete(3)

### Avaliando agente não treinado

In [ ]:
# Use a separate environement for evaluation
eval_env = gym.make("MountainCar-v0", render_mode="rgb_array")
model = DQN('MlpPolicy',
                env,
                verbose=1,
                train_freq=16,
                gradient_steps=8,
                gamma=0.99,
                exploration_fraction=0.2,
                exploration_final_eps=0.07,
                target_update_interval=100,
                learning_starts=1000,
                buffer_size=10000,
                batch_size=128,
                learning_rate=4e-3,
                policy_kwargs=dict(net_arch=[256, 256]))

# Random Agent, before training
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

# Deleta modelo
del model

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
mean_reward:-200.00 +/- 0.00


## Treinando agente e avaliando

### Inicialização

In [ ]:
# Define quantidade de treinamentos do agente
quantidades = [2e4, 5e4, 1e5, 2e5, 4e5]
qtd_teste = 10

In [ ]:
df_treinos = pd.DataFrame(index=quantidades, columns=np.arange(1, qtd_teste + 1))

### Execução

In [ ]:
for qtd in quantidades:

    # Algoritmo + Politica
    model = DQN('MlpPolicy',
                env,
                verbose=1,
                train_freq=16,
                gradient_steps=8,
                gamma=0.99,
                exploration_fraction=0.2,
                exploration_final_eps=0.07,
                target_update_interval=100,
                learning_starts=1000,
                buffer_size=10000,
                batch_size=128,
                learning_rate=4e-3,
                policy_kwargs=dict(net_arch=[256, 256]))

    # Treina agente
    model.learn(total_timesteps=qtd)

    # Salva modelo
    model.save(f"./{qtd}k_mountain_car_DQN")

    # Carrega modelo
    #model = DQN.load(f"./{qtd}k_mountain_car_DQN", env)

    # Avalia modelo
    evaluate(model, num_episodes=qtd_teste, qi=qtd)


    # Deleta modelo
    del model

    # Reseta Ambiente
    env.reset()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 200      |
|    ep_rew_mean      | -200     |
|    exploration_rate | 0.814    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 9195     |
|    time_elapsed     | 0        |
|    total_timesteps  | 800      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 200      |
|    ep_rew_mean      | -200     |
|    exploration_rate | 0.628    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 1935     |
|    time_elapsed     | 0        |
|    total_timesteps  | 1600     |
| train/              |          |
|    learning_rate    | 0.004    |
|    loss             | 0.0027   |
|    n_updates        | 296      |
-------------------------------

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 196      |
|    ep_rew_mean      | -196     |
|    exploration_rate | 0.07     |
| time/               |          |
|    episodes         | 68       |
|    fps              | 793      |
|    time_elapsed     | 16       |
|    total_timesteps  | 13350    |
| train/              |          |
|    learning_rate    | 0.004    |
|    loss             | 0.00333  |
|    n_updates        | 6176     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 197      |
|    ep_rew_mean      | -197     |
|    exploration_rate | 0.07     |
| time/               |          |
|    episodes         | 72       |
|    fps              | 791      |
|    time_elapsed     | 17       |
|    total_timesteps  | 14150    |
| train/              |          |
|    learning_rate    | 0.004    |
|    loss             | 0.109    |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 200      |
|    ep_rew_mean      | -200     |
|    exploration_rate | 0.405    |
| time/               |          |
|    episodes         | 32       |
|    fps              | 920      |
|    time_elapsed     | 6        |
|    total_timesteps  | 6400     |
| train/              |          |
|    learning_rate    | 0.004    |
|    loss             | 0.00174  |
|    n_updates        | 2696     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 200      |
|    ep_rew_mean      | -200     |
|    exploration_rate | 0.33     |
| time/               |          |
|    episodes         | 36       |
|    fps              | 895      |
|    time_elapsed     | 8        |
|    total_timesteps  | 7200     |
| train/              |          |
|    learning_rate    | 0.004    |
|    loss             | 0.000753 |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 191      |
|    ep_rew_mean      | -191     |
|    exploration_rate | 0.07     |
| time/               |          |
|    episodes         | 96       |
|    fps              | 774      |
|    time_elapsed     | 23       |
|    total_timesteps  | 18324    |
| train/              |          |
|    learning_rate    | 0.004    |
|    loss             | 0.626    |
|    n_updates        | 8664     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 191      |
|    ep_rew_mean      | -191     |
|    exploration_rate | 0.07     |
| time/               |          |
|    episodes         | 100      |
|    fps              | 774      |
|    time_elapsed     | 24       |
|    total_timesteps  | 19051    |
| train/              |          |
|    learning_rate    | 0.004    |
|    loss             | 0.559    |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 166      |
|    ep_rew_mean      | -166     |
|    exploration_rate | 0.07     |
| time/               |          |
|    episodes         | 160      |
|    fps              | 770      |
|    time_elapsed     | 36       |
|    total_timesteps  | 28495    |
| train/              |          |
|    learning_rate    | 0.004    |
|    loss             | 14.2     |
|    n_updates        | 13744    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 165      |
|    ep_rew_mean      | -165     |
|    exploration_rate | 0.07     |
| time/               |          |
|    episodes         | 164      |
|    fps              | 770      |
|    time_elapsed     | 37       |
|    total_timesteps  | 29165    |
| train/              |          |
|    learning_rate    | 0.004    |
|    loss             | 21.6     |
|    n_updates      

...

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 156      |
|    ep_rew_mean      | -156     |
|    exploration_rate | 0.07     |
| time/               |          |
|    episodes         | 1920     |
|    fps              | 616      |
|    time_elapsed     | 570      |
|    total_timesteps  | 351729   |
| train/              |          |
|    learning_rate    | 0.004    |
|    loss             | 2.28e+03 |
|    n_updates        | 175368   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 157      |
|    ep_rew_mean      | -157     |
|    exploration_rate | 0.07     |
| time/               |          |
|    episodes         | 1924     |
|    fps              | 616      |
|    time_elapsed     | 571      |
|    total_timesteps  | 352380   |
| train/              |          |
|    learning_rate    | 0.004    |
|    loss             | 3.72e+03 |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 177      |
|    ep_rew_mean      | -177     |
|    exploration_rate | 0.07     |
| time/               |          |
|    episodes         | 1984     |
|    fps              | 616      |
|    time_elapsed     | 588      |
|    total_timesteps  | 363227   |
| train/              |          |
|    learning_rate    | 0.004    |
|    loss             | 3.03e+03 |
|    n_updates        | 181112   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 176      |
|    ep_rew_mean      | -176     |
|    exploration_rate | 0.07     |
| time/               |          |
|    episodes         | 1988     |
|    fps              | 617      |
|    time_elapsed     | 589      |
|    total_timesteps  | 363880   |
| train/              |          |
|    learning_rate    | 0.004    |
|    loss             | 1.79e+03 |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 165      |
|    ep_rew_mean      | -165     |
|    exploration_rate | 0.07     |
| time/               |          |
|    episodes         | 2048     |
|    fps              | 617      |
|    time_elapsed     | 604      |
|    total_timesteps  | 373191   |
| train/              |          |
|    learning_rate    | 0.004    |
|    loss             | 4.41e+03 |
|    n_updates        | 186096   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 163      |
|    ep_rew_mean      | -163     |
|    exploration_rate | 0.07     |
| time/               |          |
|    episodes         | 2052     |
|    fps              | 617      |
|    time_elapsed     | 605      |
|    total_timesteps  | 373759   |
| train/              |          |
|    learning_rate    | 0.004    |
|    loss             | 6.02e+03 |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 151      |
|    ep_rew_mean      | -151     |
|    exploration_rate | 0.07     |
| time/               |          |
|    episodes         | 2112     |
|    fps              | 617      |
|    time_elapsed     | 619      |
|    total_timesteps  | 382959   |
| train/              |          |
|    learning_rate    | 0.004    |
|    loss             | 2.61e+04 |
|    n_updates        | 190976   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 152      |
|    ep_rew_mean      | -152     |
|    exploration_rate | 0.07     |
| time/               |          |
|    episodes         | 2116     |
|    fps              | 617      |
|    time_elapsed     | 620      |
|    total_timesteps  | 383673   |
| train/              |          |
|    learning_rate    | 0.004    |
|    loss             | 2.08e+04 |
|    n_updates      

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 154      |
|    ep_rew_mean      | -154     |
|    exploration_rate | 0.07     |
| time/               |          |
|    episodes         | 2176     |
|    fps              | 618      |
|    time_elapsed     | 634      |
|    total_timesteps  | 392688   |
| train/              |          |
|    learning_rate    | 0.004    |
|    loss             | 3.3e+04  |
|    n_updates        | 195840   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 155      |
|    ep_rew_mean      | -155     |
|    exploration_rate | 0.07     |
| time/               |          |
|    episodes         | 2180     |
|    fps              | 618      |
|    time_elapsed     | 635      |
|    total_timesteps  | 393315   |
| train/              |          |
|    learning_rate    | 0.004    |
|    loss             | 3.1e+04  |
|    n_updates      

## Avalia o agente treinado

In [ ]:
df_treinos

,1,2,3,4,5,6,7,8,9,10
20000.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0
50000.0,-184.0,-148.0,-154.0,-88.0,-148.0,-90.0,-165.0,-167.0,-164.0,-167.0
100000.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0
200000.0,-115.0,-106.0,-182.0,-121.0,-114.0,-175.0,-200.0,-113.0,-106.0,-172.0
400000.0,-146.0,-180.0,-171.0,-156.0,-145.0,-149.0,-148.0,-147.0,-90.0,-145.0


In [ ]:
df_treinos.T.astype(float).describe()

,20000.0,50000.0,100000.0,200000.0,400000.0
count,10.0,10.000000,10.0,10.000000,10.000000
mean,-200.0,-147.500000,-200.0,-140.400000,-147.700000
std,0.0,32.592603,0.0,36.990089,23.579888
min,-200.0,-184.000000,-200.0,-200.000000,-180.000000
25%,-200.0,-166.500000,-200.0,-174.250000,-154.250000
50%,-200.0,-159.000000,-200.0,-118.000000,-147.500000
75%,-200.0,-148.000000,-200.0,-113.250000,-145.250000
max,-200.0,-88.000000,-200.0,-106.000000,-90.000000


### Prepare video recording

In [ ]:
# Set up fake display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [ ]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay


def show_videos(video_path="", prefix=""):
    """
    Taken from https://github.com/eleurent/highway-env

    :param video_path: (str) Path to the folder containing videos
    :param prefix: (str) Filter the video, showing only the only starting with this prefix
    """
    html = []
    for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            """<video alt="{}" autoplay
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>""".format(
                mp4, video_b64.decode("ascii")
            )
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

We will record a video using the [VecVideoRecorder](https://stable-baselines.readthedocs.io/en/master/guide/vec_envs.html#vecvideorecorder) wrapper, you will learn about those wrapper in the next notebook.

In [ ]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv


def record_video(env_id, model, video_length=500, prefix="", video_folder="videos/"):
    """
    :param env_id: (str)
    :param model: (RL model)
    :param video_length: (int)
    :param prefix: (str)
    :param video_folder: (str)
    """
    eval_env = DummyVecEnv([lambda: gym.make("MountainCar-v0", render_mode="rgb_array")])
    # Start the video at step=0 and record 500 steps
    eval_env = VecVideoRecorder(
        eval_env,
        video_folder=video_folder,
        record_video_trigger=lambda step: step == 0,
        video_length=video_length,
        name_prefix=prefix,
    )

    obs = eval_env.reset()
    for _ in range(video_length):
        action, _ = model.predict(obs)
        obs, _, _, _ = eval_env.step(action)

    # Close the video recorder
    eval_env.close()

### Visualize trained agent



In [ ]:
#record_video("MountainCar-v0_50k", model, video_length=500, prefix="ppo-cartpole")

Saving video to /content/videos/ppo-cartpole-step-0-to-step-500.mp4
Moviepy - Building video /content/videos/ppo-cartpole-step-0-to-step-500.mp4.
Moviepy - Writing video /content/videos/ppo-cartpole-step-0-to-step-500.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/ppo-cartpole-step-0-to-step-500.mp4


### Inicializar modelo salvo e grava video

In [ ]:
for qtd in quantidades:
    model = DQN.load(f"{qtd}k_mountain_car_DQN", env)
    record_video(f"{qtd}k_mountain_car_DQN", model, video_length=500, prefix=f"ppo-{qtd}k_mountain_car_DQN")

In [ ]:
show_videos("videos", prefix="ppo")